# PREDICTING HOMELESSNESS IN AMERICA

## Data Imports

In [1]:
import pandas as pd
import numpy as np

### HUD Annual Homeless Counts

Data downloaded from: https://www.hudexchange.info/resource/3031/pit-and-hic-data-since-2007/ .
I will convert the files into a single dataframe with the COC number, total count, and year.

In [2]:
df_2016 = pd.read_csv('homeless_2016.csv')
df_2016.head()

,num,2016
0,AK-500,1105
1,AK-501,835
2,AL-500,1228
3,AL-501,623
4,AL-502,337


In [3]:
df_2016.rename({'2016':'homeless_count'}, axis=1, inplace=True)
df_2016['yr'] = 2016

In [4]:
df_2016.head()

,num,homeless_count,yr
0,AK-500,1105,2016
1,AK-501,835,2016
2,AL-500,1228,2016
3,AL-501,623,2016
4,AL-502,337,2016


In [5]:
#function to format HUD files in order to merge into a single dataframe
def import_counts(year, file):
    table = pd.read_csv(file)
    table['yr'] = year
    table.rename({str(year):'homeless_count'}, axis=1, inplace=True)
    table.dropna(axis=0, inplace=True)
    return table
    

In [6]:
df_2015 = import_counts(2015,'homeless_2015.csv')
df_2014 = import_counts(2014,'homeless_2014.csv')
df_2013 = import_counts(2013,'homeless_2013.csv')
df_2012 = import_counts(2012,'homeless_2012.csv')
df_2011 = import_counts(2011,'homeless_2011.csv')
df_2010 = import_counts(2010,'homeless_2010.csv')
df_2009 = import_counts(2009,'homeless_2009.csv')
df_2008 = import_counts(2008,'homeless_2008.csv')
df_2007 = import_counts(2007,'homeless_2007.csv')

In [7]:
merg = [df_2016, df_2015, df_2014, df_2013, df_2012, df_2011, df_2010, df_2009, df_2008, df_2007]
data = pd.concat(merg, axis=0)

In [8]:
data.shape

(3943, 3)

In [9]:
#save the dataframe to a csv file, this is the outcome data to be used in my predictive models
#data.to_csv('homeless_counts.csv')

### Importing SSI Data

Downloaded from https://www.ssa.gov/policy/docs/statcomps/ssi_sc/2016/index.html . I will create functions that compile state data by sheet in every annual excel workbook. 

In [10]:
#each workbook will have the same pages
#DC not in list, this state is missing from multiple files

sheets = ['Table 3 - Alabama',
'Table 3 - Alaska',
'Table 3 - Arizona',
'Table 3 - Arkansas',
'Table 3 - California',
'Table 3 - Colorado',
'Table 3 - Connecticut',
'Table 3 - Delaware',
'Table 3 - Florida',
'Table 3 - Georgia',
'Table 3 - Hawaii',
'Table 3 - Idaho',
'Table 3 - Illinois',
'Table 3 - Indiana',
'Table 3 - Iowa',
'Table 3 - Kansas',
'Table 3 - Kentucky',
'Table 3 - Louisiana',
'Table 3 - Maine',
'Table 3 - Maryland',
'Table 3 - Massachusetts',
'Table 3 - Michigan',
'Table 3 - Minnesota',
'Table 3 - Mississippi',
'Table 3 - Missouri',
'Table 3 - Montana',
'Table 3 - Nebraska',
'Table 3 - Nevada',
'Table 3 - New Hampshire',
'Table 3 - New Jersey',
'Table 3 - New Mexico',
'Table 3 - New York',
'Table 3 - North Carolina',
'Table 3 - North Dakota',
'Table 3 - Ohio',
'Table 3 - Oklahoma',
'Table 3 - Oregon',
'Table 3 - Pennsylvania',
'Table 3 - Rhode Island',
'Table 3 - South Carolina',
'Table 3 - South Dakota',
'Table 3 - Tennessee',
'Table 3 - Texas',
'Table 3 - Utah',
'Table 3 - Vermont',
'Table 3 - Virginia',
'Table 3 - Washington',
'Table 3 - West Virginia',
'Table 3 - Wisconsin',
'Table 3 - Wyoming']

In [11]:
# excel books from years 2009-2016 are formatted the same, create a list with years in decsending order
books = ['ssi_sc16.xlsx', 'ssi_sc15.xlsx', 'ssi_sc14.xlsx','ssi_sc13.xlsx','ssi_sc12.xlsx',
         'ssi_sc11.xlsx','ssi_sc10.xlsx','ssi_sc09.xlsx']

# excel books from years 2007-2008 are slightly different from the rest
books_2 =['ssi_sc08.xlsx', 'ssi_sc07.xlsx']

In [12]:
# function to pull relevant data from excel docs
# files for 2007 & 2008 are different, this function cannot be applied to those workbooks

def xl_clean(book, sheet):
    ssi_df = pd.read_excel(book, sheet_name = sheet) 
    ssi_df.drop(ssi_df.index[0:4], inplace=True) 
    ssi_df = ssi_df.reset_index() 

    # drop the last 8 rows 
    y = len(ssi_df.index) 
    x = y - 8
    ssi_df.drop(ssi_df.index[x:y], inplace=True)
    
    #drop columns that are blank or that contain unnecessary information
    ssi_df.drop(['Unnamed: 1', 'Unnamed: 2', 'Unnamed: 9','Unnamed: 10'], axis=1, inplace=True)
    
    #rename columns for clarity
    ssi_df.rename({'Unnamed: 3': 'total_ssi', 'Unnamed: 4': 'aged_ssi','Unnamed: 5': 'disbl_ssi',
                   'Unnamed: 6': '18_und_ssi','Unnamed: 7': '18_64_ssi',
                   'Unnamed: 8': '65_ovr_ssi'}, axis=1, inplace=True)
    
    #reset the index again and drop the unnecessary columns
    ssi_df = ssi_df.reset_index()
    ssi_df.drop(['level_0','index'], axis=1, inplace=True)
    
    #rename the first column in the sheet 
    ssi_df.rename(columns={list(ssi_df.columns)[0]:'county'}, inplace=True)
    
    #add a column with the worksheet name
    ssi_df['state'] = sheet
    return ssi_df

In [13]:
#for loop function that runs the above function through each page of one book and stores the data in a dictionary 
def xl_to_df(book, sheets):
    d = {}
    for i in range(len(sheets)):
        d[i] = xl_clean(book, sheets[i])
    return d

In [14]:
print(len(sheets))
print(len(books)) 
#the function will run through 50 pages of 8 workbooks

50
8


In [15]:
#loop through every workbook running the function that processes each page of the workbook
file = {}
for i in range(len(books)):
    file[i] = xl_to_df(books[i], sheets)

In [16]:
#function to merge the data from each workbook page into a single dataframe

def assemble(file):
    
    #make lists of 10 dictionaries or less
    list_1 = [file[0],file[1],file[2],file[3],file[4],file[5],
              file[6],file[7],file[8],file[9],file[10]]
    
    list_2 = [file[11],file[12],file[13],file[14],file[15],
              file[16],file[17],file[18],file[19],file[20]]
    
    list_3 = [file[21],file[22],file[23],file[24],file[25],
              file[26],file[27],file[28],file[29],file[30]]
    
    list_4 = [file[31],file[32],file[33],file[34],file[35],
              file[36],file[37], file[38],file[39],file[40]]
    
    list_5 = [file[41],file[42],file[43],file[44],file[45],
              file[46],file[47],file[48],file[49]]
   
    #concat the lists into dataframes and reset the index
    df_a = pd.concat(list_1, axis=0)
    df_a = df_a.reset_index()
    df_a.drop('index', axis=1, inplace=True)

    df_b = pd.concat(list_2, axis=0)
    df_b = df_b.reset_index()
    df_b.drop('index', axis=1, inplace=True)

    df_c = pd.concat(list_3, axis=0)
    df_c = df_c.reset_index()
    df_c.drop('index', axis=1, inplace=True)

    df_d = pd.concat(list_4, axis=0)
    df_d = df_d.reset_index()
    df_d.drop('index', axis=1, inplace=True)

    df_e = pd.concat(list_5, axis=0)
    df_e = df_e.reset_index()
    df_e.drop('index', axis=1, inplace=True)
    
    #create a list of the dataframes
    frames = [df_a, df_b, df_c, df_d, df_e]
    
    #concat the dataframes into a single dataframe
    df_ssi = pd.concat(frames, axis=0)
    
    return df_ssi

Test the function on a single dictionary, since the books were listed in descending order the first file will represent 2016 data.

In [17]:
df_ssi2016 = assemble(file[0])
df_ssi2016.head()

,county,total_ssi,aged_ssi,disbl_ssi,18_und_ssi,18_64_ssi,65_ovr_ssi,state
0,Autauga,1467,68,1399,236,1027,204,Table 3 - Alabama
1,Baldwin,3521,193,3328,607,2384,530,Table 3 - Alabama
2,Barbour,1469,104,1365,233,910,326,Table 3 - Alabama
3,Bibb,966,29,937,95,714,157,Table 3 - Alabama
4,Blount,1282,62,1220,121,945,216,Table 3 - Alabama


Run through the rest of the dicts.

In [18]:
df_ssi2015 = assemble(file[1])
df_ssi2014 = assemble(file[2])
df_ssi2013 = assemble(file[3])
df_ssi2012 = assemble(file[4])
df_ssi2011 = assemble(file[5])
df_ssi2010 = assemble(file[6])
df_ssi2009 = assemble(file[7])

In [19]:
#new updated function to process the 2007 & 2008 files, dropping different columns

def xl_clean_2(book, sheet):
    ssi_8 = pd.read_excel(book, sheet_name = sheet) #convert each sheet to dataframe
    ssi_8.drop(ssi_8.index[0:4], inplace=True)#drop the first 4 rows which are not useful 
    ssi_8 = ssi_8.reset_index()
   
    #drop the last 8 rows 
    y = len(ssi_8.index) 
    x = y - 8
    ssi_8.drop(ssi_8.index[x:y], inplace=True)

    #drop columns that are blank or that contain unnecessary information
    ssi_8.drop(['Unnamed: 1','Unnamed: 8', 'Unnamed: 9'], axis=1, inplace=True)

    #rename columns for clarity
    ssi_8.rename({'Unnamed: 2': 'total_ssi', 'Unnamed: 3': 'aged_ssi',
                  'Unnamed: 4': 'disbl_ssi','Unnamed: 5': '18_und_ssi',
                  'Unnamed: 6': '18_64_ssi','Unnamed: 7': '65_ovr_ssi'}, axis=1, inplace=True)
    
    #reset the index again and drop the unnecessary columns
    ssi_8 = ssi_8.reset_index()
    ssi_8.drop(['level_0','index'], axis=1, inplace=True)
    
    #rename the first column in the sheet
    ssi_8.rename(columns={list(ssi_8.columns)[0]:'county'}, inplace=True)
    
    #add a column with the worksheet name
    ssi_8['state'] = sheet
    
    return ssi_8

In [20]:
#for loop function that runs the above function through each page of one book and stores the data in a dictionary 
def xl_to_df_2(book, sheets):
    d = {}
    for i in range(len(sheets)):
        d[i] = xl_clean_2(book, sheets[i])
    return d

In [21]:
#loop through every workbook running the function that processes each page of the workbook
newfile = {}
for i in range(len(books_2)):
    newfile[i] = xl_to_df_2(books_2[i], sheets)

In [22]:
#run function on both dicts
df_ssi2008 = assemble(newfile[0])
df_ssi2007 = assemble(newfile[1])

In [24]:
#check shape to ensure same number of columns
print(df_ssi2016.shape)
print(df_ssi2015.shape)
print(df_ssi2014.shape)
print(df_ssi2013.shape)
print(df_ssi2012.shape)
print(df_ssi2011.shape)
print(df_ssi2010.shape)
print(df_ssi2009.shape)
print(df_ssi2008.shape)
print(df_ssi2007.shape)

(3149, 8)
(3149, 8)
(3151, 8)
(3150, 8)
(3150, 8)
(3150, 8)
(3142, 8)
(3164, 8)
(3116, 8)
(3096, 8)


In [26]:
df_ssi2007.head()#check a dataframe

,county,total_ssi,aged_ssi,disbl_ssi,18_und_ssi,18_64_ssi,65_ovr_ssi,state
0,Autauga,1352,123,1229,278,836,238,Table 3 - Alabama
1,Baldwin,2770,248,2522,440,1869,461,Table 3 - Alabama
2,Barbour,1720,223,1497,287,955,478,Table 3 - Alabama
3,Bibb,1070,67,1003,174,723,173,Table 3 - Alabama
4,Blount,1397,158,1239,186,881,330,Table 3 - Alabama


In [27]:
#create a list of dataframes and a list of years
ssi_files = [df_ssi2016,df_ssi2015,df_ssi2014,df_ssi2013,
         df_ssi2012,df_ssi2011,df_ssi2010, df_ssi2009,df_ssi2008, df_ssi2007]

years = ['2016', '2015', '2014', '2013', '2012', '2011', '2010', '2009', '2008','2007']

In [28]:
#function to edit state name
def state_name(var):
    x = var.split('-').pop()#split the string at the hyphen and return the last string
    return x

In [29]:
#function that loops function above, puts state and county in lowercase letters, and adds year
def location_year(df, year):
    df['state'] = df['state'].apply(lambda x: state_name(x))
    df['state'] = df['state'].apply(lambda x: str(x).lower())
    df['county'] = df['county'].apply(lambda x: str(x).lower())
    df['year'] = year
    return df

In [30]:
#loop through the list of dataframes, running abive function and storing data in dict
ssi = {}
for i in range(len(ssi_files)):
    ssi[i] = location_year(ssi_files[i], years[i])
    

In [31]:
#create one large dataframe 
ssi_data = [ssi[0], ssi[1], ssi[2], ssi[3], ssi[4], ssi[5], ssi[6], ssi[7], ssi[8], ssi[9]]
ssi_df = pd.concat(ssi_data, axis=0)

In [32]:
ssi_df.shape

(31417, 9)

In [33]:
#save the dataframe to a csv file
#ssi_df.to_csv('ssi_df.csv')

### Unemployment Data by County 

Compiled Bureau of Labor Stats data found on Kaggle: 
https://www.kaggle.com/jayrav13/unemployment-by-county-us/data

In [34]:
un_df = pd.read_csv('ALL_unemployment_county.csv')

In [35]:
un_df.head()

,Year,Month,State,County,Rate
0,2015,February,Mississippi,Newton County,6.1
1,2015,February,Mississippi,Panola County,9.4
2,2015,February,Mississippi,Monroe County,7.9
3,2015,February,Mississippi,Hinds County,6.1
4,2015,February,Mississippi,Kemper County,10.6


In [36]:
#format the dataframe by renaming columns and making strings lowercase
un_df.columns = map(str.lower, un_df.columns)
un_df.rename({'rate': 'unemploy_rate'}, axis=1, inplace=True)
un_df.county = un_df.county.apply(lambda x: x.lower())
un_df.state = un_df.state.apply(lambda x: x.lower())

In [37]:
#function to split the county name and remove the last word "county"
def split(value):
    value = value.split(" ")[:-1]
    return ' '.join(value)

In [38]:
#loop through all counties in column
un_df.county = un_df.county.apply(lambda x: split(x))

In [39]:
#create separate dataframes of years needed
un_16 = un_df[un_df.year == 2016]
un_15 = un_df[un_df.year == 2015]
un_14 = un_df[un_df.year == 2014]
un_13 = un_df[un_df.year == 2013]
un_12 = un_df[un_df.year == 2012]
un_11 = un_df[un_df.year == 2011]
un_10 = un_df[un_df.year == 2010]
un_09 = un_df[un_df.year == 2009]
un_08 = un_df[un_df.year == 2008]
un_07 = un_df[un_df.year == 2007]

In [40]:
#create a list of dataframes to loop through
unemploy = [un_16, un_15, un_14, un_13, un_12, un_11, un_10, un_09, un_08, un_07]

In [41]:
#do not need monthly data, take the average of all monthly to get annual data
un_16 = un_16.groupby(['state','county']).mean()

In [42]:
#loop through all years to get average annual unemployment rate by grouping together county and state and taking the mean
for i in range(len(unemploy)): 
    unemploy[i] = unemploy[i].groupby(['state','county']).mean()
    unemploy[i].reset_index(inplace=True)
    unemploy[i].unemploy_rate = unemploy[i].unemploy_rate.round(2)


In [43]:
#combine all dataframes into one large dataframe
emp_df = pd.concat(unemploy, axis=0)
emp_df.head()

,state,county,year,unemploy_rate
0,alabama,autauga,2016,5.21
1,alabama,baldwin,2016,5.35
2,alabama,barbour,2016,8.53
3,alabama,bibb,2016,6.51
4,alabama,blount,2016,5.43


In [46]:
#save the dataframe to a csv file
#emp_df.to_csv('unemployment_data.csv')